Copyright 2018 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

In [1]:
pip install scikeras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install -q tensorflow-datasets tensorflow

In [3]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

2022-12-18 18:01:02.919088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 18:01:03.205300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-18 18:01:03.205316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-18 18:01:03.237324: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-18 18:01:04.017880: W tensorflow/stream_executor/platform/de

In [4]:
pip install keras_cv

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 KB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
import keras_cv
import numpy as np
import random
#'train' is the only available split
ds = tfds.load('colorectal_histology', split='train', shuffle_files=True)


2022-12-18 18:01:08.297072: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-18 18:01:08.297134: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Momo-linux): /proc/driver/nvidia/version does not exist
2022-12-18 18:01:08.299322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Resizing, Rescaling
from keras_cv.layers import Grayscale

In [7]:
X = np.array([row['image'] for row in ds])


In [8]:
y = np.array([row['label'] for row in ds])
y = keras.utils.to_categorical(y)
z = y[0][:]

In [9]:
y = y[:,0]

# Data augmentation for tumors

In [10]:
tumor_images = X[np.where(y == 1)]
tumor_images.shape

(625, 150, 150, 3)

In [11]:
new_tumors = []
while len(new_tumors)<5000:
  random_index = random.randint(0,len(tumor_images)-1)
  random_tumor = tumor_images[random_index]
  rotated_tumor = tf.keras.preprocessing.image.random_rotation(x = random_tumor,
                                                              row_axis=0,
                                                              col_axis=1,
                                                              channel_axis=2,
                                                              rg = 360)
  new_tumors.append(rotated_tumor) 
  y = np.append(y, 1)

ary_tumors = np.array(new_tumors)

In [ ]:
ary_tumors = np.array(new_tumors)
X = np.concatenate((X,ary_tumors))


Visual proof of the working rotation

In [ ]:
from PIL import Image
import numpy as np

w, h = 512, 512
data = np.zeros((h, w, 3), dtype=np.uint8)
data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(
    tf.keras.preprocessing.image.random_rotation(x = X[-1],
                                                  row_axis=0,
                                                  col_axis=1,
                                                  channel_axis=2,
                                                  rg = 360),
                       'RGB')
img.save('my.png')
img.show()

In [11]:
X.shape

(10000, 150, 150, 3)

# Grey scaling



In [14]:
def to_grey_scale(X):
  return np.dot(X[...,:3], [.3, .6, .1])

#to_grey_scale(X)

array([[[220.6, 214.6, 218.9, ..., 241.8, 241.1, 243.1],
        [217.6, 215.6, 213.9, ..., 242.1, 242.1, 243.1],
        [215.1, 218.1, 219.8, ..., 242.1, 242.1, 243.1],
        ...,
        [241.9, 241.9, 241.9, ..., 241. , 241.3, 241.3],
        [241.9, 241.9, 241.9, ..., 241. , 241.3, 241.3],
        [241.9, 241.9, 241.9, ..., 241. , 241.3, 241.3]],

       [[241.2, 242.2, 243.2, ..., 244.1, 243.9, 243.9],
        [241.2, 242.2, 243.2, ..., 244.1, 243.9, 243.9],
        [241.2, 242.2, 243.2, ..., 244.1, 243.9, 243.9],
        ...,
        [242.3, 242.3, 242.3, ..., 243. , 242.8, 242.8],
        [242.3, 242.3, 242.3, ..., 244. , 243.8, 243.8],
        [242.3, 242.3, 242.3, ..., 244. , 243.8, 243.8]],

       [[ 97.4,  99.4, 122.4, ..., 160.3, 172.6, 185.6],
        [104.1, 108.1, 122.3, ..., 163.3, 178.9, 191.9],
        [111.6, 115.6, 117.3, ..., 174.8, 182.9, 190.9],
        ...,
        [153.7, 146.7, 123.2, ..., 102.8, 110.9, 110.9],
        [134.5, 120.5, 106.5, ..., 100.2, 105

In [15]:
grayed_images = []
for image in X:
  np.dot(image)

TypeError: ignored

In [16]:
to_grayscale = keras_cv.layers.preprocessing.Grayscale()
augmented_images = to_grayscale(X)

# Manual shuffle data

In [17]:
data = [(augmented_images[i],y[i]) for i in range(len(y))]

In [18]:
random.shuffle(data)

In [19]:
augmented_images = np.array([data[i][0] for i in range(len(data))])
y = np.array([data[i][1] for i in range(len(data))])


In [20]:
(y[1],y[2],y[3])


(1.0, 1.0, 0.0)

# Manual train test split

In [21]:
assert len(augmented_images) == len(y)
train_test_ratio = .8
split_index = int(len(augmented_images)*train_test_ratio)

X_train = augmented_images[:split_index]
y_train = y[:split_index]
X_test = augmented_images[split_index:]
y_test = y[split_index:]


# Model

In [22]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(optimizer__learning_rate=learn_rate)

In [23]:
def create_model():
  return keras.models.Sequential([
    tf.keras.layers.Rescaling(scale = 1/255, input_shape=(150,150,1)),
    #keras_cv.layers.preprocessing.Grayscale(),
    Conv2D(16,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(32,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding="same",activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1,activation='sigmoid'),
])


In [24]:
model = KerasClassifier(model=create_model, loss="binary_crossentropy", optimizer="SGD", epochs=100, batch_size=10, verbose=0)

In [25]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [77]:
model.compile(tf.keras.optimizers.SGD(learning_rate=0.0001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.Recall(), "accuracy"]
)

In [78]:
(X.shape,y.shape)

((10000, 150, 150, 3), (10000,))

In [79]:
#model.fit(augmented_images,y,batch_size=32, epochs=3, validation_split=.2)
model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          epochs = 10)

Epoch 1/10
250/250 [==============================] - 6s 20ms/step - loss: 0.6858 - recall_8: 1.0000 - accuracy: 0.5592 - val_loss: 0.6821 - val_recall_8: 1.0000 - val_accuracy: 0.5755
Epoch 2/10
250/250 [==============================] - 4s 18ms/step - loss: 0.6858 - recall_8: 1.0000 - accuracy: 0.5592 - val_loss: 0.6821 - val_recall_8: 1.0000 - val_accuracy: 0.5755
Epoch 3/10
250/250 [==============================] - 4s 18ms/step - loss: 0.6858 - recall_8: 1.0000 - accuracy: 0.5592 - val_loss: 0.6821 - val_recall_8: 1.0000 - val_accuracy: 0.5755
Epoch 4/10
250/250 [==============================] - 4s 18ms/step - loss: 0.6858 - recall_8: 1.0000 - accuracy: 0.5592 - val_loss: 0.6821 - val_recall_8: 1.0000 - val_accuracy: 0.5755
Epoch 5/10
250/250 [==============================] - 4s 18ms/step - loss: 0.6858 - recall_8: 1.0000 - accuracy: 0.5592 - val_loss: 0.6821 - val_recall_8: 1.0000 - val_accuracy: 0.5755
Epoch 6/10
250/250 [==============================] - 4s 18ms/step - loss: 

In [59]:
sum(np.where(model.predict(augmented_images) > .5))

313/313 [==============================] - 2s 6ms/step


array([   0,    1,    2, ..., 9997, 9998, 9999])